In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense

In [11]:

dataset = pd.read_csv("data_for_traing_ml_models.csv")

#####################

'''
7-feature dataset is run on Simple Linear Regression, LASSO, Ridge, Random Forest and Neural Network models for the prediction of Heat of Formation of MXenes.
'''

'\n7-feature dataset is run on Simple Linear Regression, LASSO, Ridge, Random Forest and Neural Network models for the prediction of Heat of Formation of MXenes.\n'

In [12]:
# take a look at the dataset
dataset.head()
dataset.describe()

,Work_function,Heat_of_formation,Energy_above_convex_hull,Length(x),Radius_of_orbital_of_metal,Radius_of_orbital_of_support,Radius_of_orbital_of_termination,Ionisation_potential_of_metal,Ionisation_potential_of_support,Ionisation_potential_of_termination,Electronegativity_of_metal,Electronegativity_of_support,Electronegativity_of_termination,Electron_affinity_of_metal,Electron_affinity_of_support,Electron_affinity_of_termination
count,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000,275.000000
mean,4.264700,-1.062837,0.315639,311.577091,65.518545,54.219636,45.983636,6.980777,12.891248,12.719608,1.601527,2.794109,2.951018,0.394964,0.598422,1.755294
std,1.530484,0.835387,0.228956,20.841980,16.428133,5.409809,13.987596,0.518915,1.639873,3.765861,0.333816,0.245445,0.903641,0.364981,0.667210,1.099077
min,1.347678,-2.984132,0.000000,262.100000,38.900000,48.800000,36.000000,6.217000,11.260300,6.217000,1.220000,2.550000,1.220000,-0.500000,-0.070000,-0.500000
25%,2.792242,-1.646810,0.162571,294.700000,48.900000,48.800000,38.900000,6.633900,11.260300,7.980000,1.330000,2.550000,2.360000,0.189000,-0.070000,0.894000
50%,4.537063,-1.079087,0.268825,312.500000,74.600000,59.600000,41.400000,6.825000,11.260300,13.017000,1.540000,2.550000,2.820000,0.323000,1.262000,1.439000
75%,5.349280,-0.483631,0.424251,324.000000,79.000000,59.600000,41.400000,7.264100,14.534100,13.618000,1.630000,3.040000,3.440000,0.749000,1.262000,1.827000
max,7.271891,0.957153,1.138551,378.300000,86.500000,59.600000,86.500000,7.980000,14.534100,17.422800,2.360000,3.040000,3.980000,0.894000,1.262000,3.401000


In [13]:
cdf = dataset[['Work_function','Heat_of_formation','Energy_above_convex_hull','Length(x)','Radius_of_orbital_of_metal','Radius_of_orbital_of_support','Radius_of_orbital_of_termination','Ionisation_potential_of_metal','Ionisation_potential_of_support','Ionisation_potential_of_termination','Electronegativity_of_metal','Electronegativity_of_support','Electronegativity_of_termination','Electron_affinity_of_metal','Electron_affinity_of_support','Electron_affinity_of_termination']]
cdf.head(9)

,Work_function,Heat_of_formation,Energy_above_convex_hull,Length(x),Radius_of_orbital_of_metal,Radius_of_orbital_of_support,Radius_of_orbital_of_termination,Ionisation_potential_of_metal,Ionisation_potential_of_support,Ionisation_potential_of_termination,Electronegativity_of_metal,Electronegativity_of_support,Electronegativity_of_termination,Electron_affinity_of_metal,Electron_affinity_of_support,Electron_affinity_of_termination
0,2.271378,-1.345952,0.242880,330.4,82.9,59.6,41.4,6.8250,11.2603,13.0170,1.30,2.55,2.82,0.178,1.262,1.827
1,1.588009,-1.199954,0.452744,335.0,53.9,59.6,41.4,6.5614,11.2603,13.0170,1.36,2.55,2.82,0.189,1.262,1.827
2,2.209840,-0.317344,0.473693,290.6,74.6,59.6,41.4,7.9800,11.2603,13.0170,2.36,2.55,2.82,0.816,1.262,1.827
3,2.463148,-0.594739,0.327053,285.0,70.2,48.8,41.4,7.0942,14.5341,13.0170,2.16,3.04,2.82,0.749,-0.070,1.827
4,2.329750,-1.597694,0.163033,298.9,48.9,48.8,41.4,6.8282,14.5341,13.0170,1.54,3.04,2.82,0.087,-0.070,1.827
5,2.429486,-1.645322,0.315790,320.4,79.0,48.8,41.4,6.6339,14.5341,13.0170,1.33,3.04,2.82,0.427,-0.070,1.827
6,6.244289,-0.755420,0.290092,293.1,38.9,59.6,36.0,7.4340,11.2603,17.4228,1.55,2.55,3.98,-0.500,1.262,3.401
7,4.424606,-1.265524,0.269073,316.7,78.4,59.6,36.0,7.8900,11.2603,17.4228,1.50,2.55,3.98,0.323,1.262,3.401
8,4.468438,-1.715968,0.168985,348.9,86.5,59.6,36.0,6.2170,11.2603,17.4228,1.22,2.55,3.98,0.308,1.262,3.401


In [14]:
###################### Simple Linear Regression

regr = LinearRegression()

# Assuming 'HoF', 'E_Hull', 'Length(x)', ..., are your features
features = ['Radius_of_orbital_of_metal','Radius_of_orbital_of_termination', 'Ionisation_potential_of_metal','Ionisation_potential_of_support','Electronegativity_of_metal','Electronegativity_of_termination','Electron_affinity_of_metal']
target = ['Heat_of_formation']

# Split the data into train and test sets
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

# Train set
train_x = np.asanyarray(train[features])
train_y = np.asanyarray(train[target])

# Test set
test_x = np.asanyarray(test[features])
test_y = np.asanyarray(test[target])

# Create and train the Linear Regression model
regr.fit(train_x, train_y)

# Predictions on train and test sets
train_y_hat = regr.predict(train_x)
test_y_hat = regr.predict(test_x)

# Calculate MSE, MAE, and R2 for the train set
train_mse = mean_squared_error(train_y, train_y_hat)
train_mae = mean_absolute_error(train_y, train_y_hat)
train_r2 = r2_score(train_y, train_y_hat)

# Calculate MSE, MAE, and R2 for the test set
test_mse = mean_squared_error(test_y, test_y_hat)
test_mae = mean_absolute_error(test_y, test_y_hat)
test_r2 = r2_score(test_y, test_y_hat)

# Print the results
print('Linear Regression\n')
print("Train set metrics:")
#print("MSE: %.2f" % train_mse)
print("MAE: %.2f" % train_mae)
print("R2 Score: %.2f" % train_r2)
print('LinearRegression\n')
print("\nTest set metrics:")
#print("MSE: %.2f" % test_mse)
print("MAE: %.2f" % test_mae)
print("R2 Score: %.2f" % test_r2)


Linear Regression

Train set metrics:
MAE: 0.30
R2 Score: 0.79
LinearRegression


Test set metrics:
MAE: 0.29
R2 Score: 0.84


In [15]:
####################### LASSO Regression


# Initialize LASSO model
regr1 = Lasso()
# Create and train the LASSO Regression model
regr1.fit(train_x, train_y)

# Predictions on train and test sets
train_y_hat1 = regr1.predict(train_x)
test_y_hat1 = regr1.predict(test_x)


# Calculate MSE, MAE, and R2 for the train set
train_mse1 = mean_squared_error(train_y, train_y_hat1)
train_mae1 = mean_absolute_error(train_y, train_y_hat1)
train_r2_1 = r2_score(train_y, train_y_hat1)

# Calculate MSE, MAE, and R2 for the test set
test_mse1 = mean_squared_error(test_y, test_y_hat1)
test_mae1 = mean_absolute_error(test_y, test_y_hat1)
test_r2_1 = r2_score(test_y, test_y_hat1)

# Print the results
print('LASSO Regression')
print("Train set metrics:")
#print("MSE: %.2f" % train_mse1)
print("MAE: %.2f" % train_mae1)
print("R2 Score: %.2f" % train_r2_1)

print("\nTest set metrics:")
#print("MSE: %.2f" % test_mse1)
print("MAE: %.2f" % test_mae1)
print("R2 Score: %.2f" % test_r2_1)





LASSO Regression
Train set metrics:
MAE: 0.54
R2 Score: 0.31

Test set metrics:
MAE: 0.58
R2 Score: 0.37


In [16]:
####################### Ridge Regression



# Initialize Ridge Regression
regr2 = Ridge()
# Create and train the Ridge Regression model
regr2.fit(train_x, train_y)

# Predictions on train and test sets
train_y_hat2 = regr2.predict(train_x)
test_y_hat2 = regr2.predict(test_x)


# Calculate MSE, MAE, and R2 for the train set
train_mse2 = mean_squared_error(train_y, train_y_hat2)
train_mae2 = mean_absolute_error(train_y, train_y_hat2)
train_r2_2 = r2_score(train_y, train_y_hat2)

# Calculate MSE, MAE, and R2 for the test set
test_mse2 = mean_squared_error(test_y, test_y_hat2)
test_mae2 = mean_absolute_error(test_y, test_y_hat2)
test_r2_2 = r2_score(test_y, test_y_hat2)

# Print the results
print('Ridge Regression\n')
print("Train set metrics:")
#print("MSE: %.2f" % train_mse2)
print("MAE: %.2f" % train_mae2)
print("R2 Score: %.2f" % train_r2_2)

print("\nTest set metrics:")
#print("MSE: %.2f" % test_mse2)
print("MAE: %.2f" % test_mae2)
print("R2 Score: %.2f" % test_r2_2)





Ridge Regression

Train set metrics:
MAE: 0.30
R2 Score: 0.79

Test set metrics:
MAE: 0.29
R2 Score: 0.84


In [17]:
###################### Random Forest Regression



# Initialize Random Forest Regression
regr3 = RandomForestRegressor()
# Create and train the Random Forest Regression model
regr3.fit(train_x, train_y)

# Predictions on train and test sets
train_y_hat3 = regr3.predict(train_x)
test_y_hat3 = regr3.predict(test_x)


# Calculate MSE, MAE, and R2 for the train set
train_mse3 = mean_squared_error(train_y, train_y_hat3)
train_mae3 = mean_absolute_error(train_y, train_y_hat3)
train_r2_3 = r2_score(train_y, train_y_hat3)

# Calculate MSE, MAE, and R2 for the test set
test_mse3 = mean_squared_error(test_y, test_y_hat3)
test_mae3 = mean_absolute_error(test_y, test_y_hat3)
test_r2_3 = r2_score(test_y, test_y_hat3)

# Print the results
print('Random Forest Regression \n')
print("Train set metrics:")
#print("MSE: %.2f" % train_mse3)
print("MAE: %.2f" % train_mae3)
print("R2 Score: %.2f" % train_r2_3)

print("\nTest set metrics:")
#print("MSE: %.2f" % test_mse3)
print("MAE: %.2f" % test_mae3)
print("R2 Score: %.2f" % test_r2_3)






Random Forest Regression 

Train set metrics:
MAE: 0.15
R2 Score: 0.94

Test set metrics:
MAE: 0.23
R2 Score: 0.90


/home/azaan/anaconda3/envs/data_science/lib/python3.9/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


################## Neural Network Regression

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization

# Assuming 'dataset' is already loaded
dataset_columns = dataset.columns
columns_to_exclude = ['Formula', 'M', 'X', 'T', 'Work_function', 'Heat_of_formation', 'Energy_above_convex_hull', 'Length(x)','Radius_of_orbital_of_support','Electron_affinity_of_termination','Electronegativity_of_support','Ionisation_potential_of_termination','Electron_affinity_of_support']

# Preprocess the data
predictors = dataset.drop(columns=columns_to_exclude)
target = dataset['Heat_of_formation']
predictors_norm = (predictors - predictors.mean()) / predictors.std()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.2, random_state=42)

# Get the number of input features
n_cols = predictors_norm.shape[1]

def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(n_cols,), kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu', kernel_regularizer='l2'))
    model.add(Dense(1))
    optimizer = Adamax(learning_rate=0.002)
    model.compile(optimizer=optimizer, loss='mae')
    return model


# Build the model
model = regression_model()

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Fit the model
model.fit(X_train, y_train, batch_size=32, validation_split=0.2, epochs=600, verbose=1, callbacks=[early_stopping])

# Make predictions on the training set
train_predictions = model.predict(X_train)

# Make predictions on the testing set
test_predictions = model.predict(X_test)

# Calculate R2 score for training and testing sets
r2_train = r2_score(y_train, train_predictions)
r2_test = r2_score(y_test, test_predictions)

# Calculate MAE for training and testing sets
mae_train = mean_absolute_error(y_train, train_predictions)
mae_test = mean_absolute_error(y_test, test_predictions)

# Print the results
print(f'Training R2 Score: {r2_train}')
print(f'Testing R2 Score: {r2_test}')
print(f'Training Mean Absolute Error: {mae_train}')
print(f'Testing Mean Absolute Error: {mae_test}')


/home/azaan/anaconda3/envs/data_science/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - loss: 7.2288 - val_loss: 7.1887
Epoch 2/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.8997 - val_loss: 7.0474
Epoch 3/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 6.7287 - val_loss: 6.9102
Epoch 4/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.5395 - val_loss: 6.7931
Epoch 5/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.4719 - val_loss: 6.6932
Epoch 6/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.3982 - val_loss: 6.5962
Epoch 7/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.2199 - val_loss: 6.5040
Epoch 8/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.2455 - val_loss: 6.3866
Epoch 9/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.1242 - val_loss: 6.2691
Epoch 10/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6.0262 - val_loss: 6.1629
Epoch 11/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 5.9161 - val_loss: 6.0784
Epoch 12/600
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5.8915 - val_los